In [1]:
# written on 11/15/23 by @caramnix 
import pandas as pd
import numpy as np

In [2]:
df= pd.read_csv("/Users/caranix/Library/CloudStorage/OneDrive-TheOhioStateUniversity/CS MS/CSPAN/testing_117/117th_raw_query_match_data_date.csv")


In [3]:
def find_string(data):
    data= data.lower()
    
    data = data.replace('-', ' ')      
    return ("one minute" in data)

In [4]:
df["one_minute_bin"] = df["text"].apply(find_string)

In [5]:
one_minute_df = df[df["one_minute_bin"] ==True]

In [6]:
one_minute_df['index_col']= one_minute_df.index

/var/folders/09/l3m63nj90fg1l2jshjp_hd5w0000gn/T/ipykernel_69474/376822450.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one_minute_df['index_col']= one_minute_df.index


In [7]:
one_minute_df.head()

,i,url,time,name,party-state,text,congress,date,one_minute_bin,index_col
50,0,https://www.c-span.org/video/?507670-1&action=...,02:43:12,Jim McGovern,D-MA,"jim mcgovern, d-mai thank the speaker. i thank...",116,"JANUARY 4, 2021",True,50
51,0,https://www.c-span.org/video/?507670-1&action=...,02:45:12,NaN,NaN,the gentleman is recognized for one minute.,116,"JANUARY 4, 2021",True,51
60,0,https://www.c-span.org/video/?507670-1&action=...,02:50:34,Jim McGovern,D-MA,"jim mcgovern, d-mai'm happy to yieldone minute...",116,"JANUARY 4, 2021",True,60
70,0,https://www.c-span.org/video/?507670-1&action=...,02:56:01,Jim McGovern,D-MA,"jim mcgovern, d-mathank you, madam speaker. if...",116,"JANUARY 4, 2021",True,70
82,0,https://www.c-span.org/video/?507670-1&action=...,03:01:49,Tom Cole,R-OK,"tom cole, r-okthank you, madam speaker. i yiel...",116,"JANUARY 4, 2021",True,82


In [ ]:
# mentions one minute, then the NEXT one is the one-minute speech 

# mentions one minute, then the next statement is "without objection, the gentlewoman is recogni..." and it's the NEXT one with the speech 

# idea: everytime one-minute mentioned, take next 5 lines.
# if the next line has "without objection", go to the next line! 
# then if the next line has a named speaker, this is the one minute speech-
# record "start time" / time stamp 
# also check to see if the next (2) lines have same speaker,
# lump them all into one  if so, since these are the same speech. 
# also take time stamp from *next* line after speech done, add as row "end time" 



In [146]:
# preininitalize columns of df 
#colnames= ["url", "start_time", "end_time", "name", "party_state", "text", 'congress', "date"]
#new_df = pd.DataFrame(columns = colnames)

In [147]:
#for i in range(0, len(one_minute_df)): 
#    index= int(one_minute_df.iloc[i].index_col)
#    
#    if pd.isna(df.iloc[index+1]["party-state"]) == False: #if name of next line is not NA... 
#        url = df.iloc[index+1].url 
#        start_time =  df.iloc[index+1].time
#        name = df.iloc[index+1]["name"]  
#        party_state =  df.iloc[index+1]["party-state"]
#        congress = 116 
#        date= df.iloc[index+1].date
        
        #this should be 1 min speech
#        if df.iloc[index+1]["name"] == df.iloc[index+2]["name"]: #if name matches for next two, combine 
#            text_of_speech= df.iloc[index+1].text + df.iloc[index+2].text
#            end_time = df.iloc[index+3].time 
#
#        else: 
#            text_of_speech= df.iloc[index+1].text
#            end_time = df.iloc[index+2].time
#    
#    
#        list_i = [url, start_time, end_time, name, party_state,text_of_speech, congress, date]
#        new_df.loc[len(new_df)] = list_i
#    else: #if name is na 
#       pass 

In [8]:
# preininitalize columns of df 
colnames= ["url", "start_time", "end_time", "name", "party_state", "text", 'congress', "date"]
new_df = pd.DataFrame(columns = colnames)

In [9]:
ct=0 
forgotten_indicies= []
for i in range(0, len(one_minute_df)): 
    index= int(one_minute_df.iloc[i].index_col)
    
    if pd.isna(df.iloc[index+1]["name"]) == False: #if name of next line is not NA... 
        url = df.iloc[index+1].url 
        start_time =  df.iloc[index+1].time
        name = df.iloc[index+1]["name"]  
        party_state =  df.iloc[index+1]["party-state"]
        congress = 116 
        date= df.iloc[index+1].date
        
        #this should be 1 min speech
        if df.iloc[index+1]["name"] == df.iloc[index+2]["name"]: #if name matches for next two, combine 
            text_of_speech= df.iloc[index+1].text + df.iloc[index+2].text
            end_time = df.iloc[index+3].time 
            ct+=1

        else: 
            text_of_speech= df.iloc[index+1].text
            end_time = df.iloc[index+2].time
    
    
        list_i = [url, start_time, end_time, name, party_state,text_of_speech, congress, date]
        new_df.loc[len(new_df)] = list_i
    else: #if name of next is na 
        
        # if it says is recognized and doesn't say false, i.e. won't be caught otherwise 
        if ("is recognized" in df.iloc[index+1].text) and (df.iloc[index+1].one_minute_bin == False): 
            
            #if the name isn't NA for the next one 
            if pd.isna(df.iloc[index+2]["name"]) == False:
                url = df.iloc[index+2].url 
                start_time =  df.iloc[index+2].time
                name = df.iloc[index+2]["name"]  
                party_state =  df.iloc[index+2]["party-state"]
                congress = 116 
                date= df.iloc[index+2].date
                
                #this should be 1 min speech
                if df.iloc[index+2]["name"] == df.iloc[index+3]["name"]: #if name matches for next two, combine 
                    text_of_speech= df.iloc[index+2].text + df.iloc[index+3].text
                    end_time = df.iloc[index+4].time 
                    ct+=1
                else: 
                    text_of_speech= df.iloc[index+2].text
                    end_time = df.iloc[index+3].time
            list_i = [url, start_time, end_time, name, party_state,text_of_speech, congress, date]
            new_df.loc[len(new_df)] = list_i
            
        else: 
            #also appends ones who get caught the next go around, i.e. because one_minute_bin is True 
            forgotten_indicies.append(i)
                
            
    
    #mini_df= df.iloc[index: index+5]


In [10]:
ct # 1,500 times we combined 

545

In [180]:
#forgotten_indicies[0:5]

In [181]:
#one_minute_df.iloc[5]

In [182]:
#df[one_minute_df.iloc[5].index_col: one_minute_df.iloc[5].index_col+5]

In [11]:
new_df

,url,start_time,end_time,name,party_state,text,congress,date
0,https://www.c-span.org/video/?507670-1&action=...,02:45:14,02:46:33,Charlie Crist Jr.,D-FL,"charlie crist jr., d-flthank you, madam speake...",116,"JANUARY 4, 2021"
1,https://www.c-span.org/video/?507670-1&action=...,02:50:42,02:52:06,Stacey Plaskett,D-VI,"stacey plaskett, d-vithank you, madam speaker....",116,"JANUARY 4, 2021"
2,https://www.c-span.org/video/?507670-1&action=...,02:56:59,02:57:56,Mary Gay Scanlon,D-PA,"mary gay scanlon, d-pain the two years i have ...",116,"JANUARY 4, 2021"
3,https://www.c-span.org/video/?507670-1&action=...,03:02:03,03:03:13,Dan Newhouse,R-WA,"dan newhouse, r-wathank you, madam speaker. th...",116,"JANUARY 4, 2021"
4,https://www.c-span.org/video/?507670-1&action=...,03:03:23,03:04:41,Sheila Jackson Lee,D-TX,"sheila jackson lee, d-txi thank the distinguis...",116,"JANUARY 4, 2021"
...,...,...,...,...,...,...,...,...
3089,https://www.c-span.org/video/?525045-1&action=...,00:14:31,00:15:39,Bill Huizenga,R-MI,"bill huizenga, r-mimr. speaker, as the 117th c...",116,"DECEMBER 23, 2022"
3090,https://www.c-span.org/video/?525045-1&action=...,00:16:03,00:16:41,Tim Burchett,R-TN,"tim burchett, r-tnmr. speaker, i rise in stron...",116,"DECEMBER 23, 2022"
3091,https://www.c-span.org/video/?525045-1&action=...,00:17:45,00:38:25,Steny Hoyer,D-MD,"steny hoyer, d-mdrevise and extend my remarks....",116,"DECEMBER 23, 2022"
3092,https://www.c-span.org/video/?525045-1&action=...,00:39:07,00:39:39,Alex Mooney,R-WV,"alex mooney, r-wvthank you, mr. speaker. i ris...",116,"DECEMBER 23, 2022"


In [12]:
def remove_name_from_text(row):    
    name_party= row['name'].lower() + ", " + row["party_state"].lower() 
    without_name= row['text'].replace(name_party, " ")

    return(without_name.replace("show full text show less text", " "))
    

In [15]:
cleaned_text=[]
for i in range(0,len(new_df)): 
    try:
        cleaned_text.append(remove_name_from_text(new_df.iloc[i]))
    except AttributeError: 
        print("here")
        cleaned_text.append("")
    
    
    

here
here


In [16]:
new_df['cleaned_text']= cleaned_text

In [17]:
new_df

,url,start_time,end_time,name,party_state,text,congress,date,cleaned_text
0,https://www.c-span.org/video/?507670-1&action=...,02:45:14,02:46:33,Charlie Crist Jr.,D-FL,"charlie crist jr., d-flthank you, madam speake...",116,"JANUARY 4, 2021","thank you, madam speaker. thank you, mr. chai..."
1,https://www.c-span.org/video/?507670-1&action=...,02:50:42,02:52:06,Stacey Plaskett,D-VI,"stacey plaskett, d-vithank you, madam speaker....",116,"JANUARY 4, 2021","thank you, madam speaker. as we begin the 117..."
2,https://www.c-span.org/video/?507670-1&action=...,02:56:59,02:57:56,Mary Gay Scanlon,D-PA,"mary gay scanlon, d-pain the two years i have ...",116,"JANUARY 4, 2021","in the two years i have been here, every moti..."
3,https://www.c-span.org/video/?507670-1&action=...,03:02:03,03:03:13,Dan Newhouse,R-WA,"dan newhouse, r-wathank you, madam speaker. th...",116,"JANUARY 4, 2021","thank you, madam speaker. this rules package ..."
4,https://www.c-span.org/video/?507670-1&action=...,03:03:23,03:04:41,Sheila Jackson Lee,D-TX,"sheila jackson lee, d-txi thank the distinguis...",116,"JANUARY 4, 2021",i thank the distinguished chair and all of th...
...,...,...,...,...,...,...,...,...,...
3089,https://www.c-span.org/video/?525045-1&action=...,00:14:31,00:15:39,Bill Huizenga,R-MI,"bill huizenga, r-mimr. speaker, as the 117th c...",116,"DECEMBER 23, 2022","mr. speaker, as the 117th congress comes to a..."
3090,https://www.c-span.org/video/?525045-1&action=...,00:16:03,00:16:41,Tim Burchett,R-TN,"tim burchett, r-tnmr. speaker, i rise in stron...",116,"DECEMBER 23, 2022","mr. speaker, i rise in strong opposition to h..."
3091,https://www.c-span.org/video/?525045-1&action=...,00:17:45,00:38:25,Steny Hoyer,D-MD,"steny hoyer, d-mdrevise and extend my remarks....",116,"DECEMBER 23, 2022","revise and extend my remarks. mr. speaker, we..."
3092,https://www.c-span.org/video/?525045-1&action=...,00:39:07,00:39:39,Alex Mooney,R-WV,"alex mooney, r-wvthank you, mr. speaker. i ris...",116,"DECEMBER 23, 2022","thank you, mr. speaker. i rise today to share..."


In [18]:
new_df.to_csv("/Users/caranix/Library/CloudStorage/OneDrive-TheOhioStateUniversity/CS MS/CSPAN/testing_117/cleaned_transcript_data_117th.csv")




In [20]:
new_df.drop_duplicates(subset=['cleaned_text'])

,url,start_time,end_time,name,party_state,text,congress,date,cleaned_text
0,https://www.c-span.org/video/?507670-1&action=...,02:45:14,02:46:33,Charlie Crist Jr.,D-FL,"charlie crist jr., d-flthank you, madam speake...",116,"JANUARY 4, 2021","thank you, madam speaker. thank you, mr. chai..."
1,https://www.c-span.org/video/?507670-1&action=...,02:50:42,02:52:06,Stacey Plaskett,D-VI,"stacey plaskett, d-vithank you, madam speaker....",116,"JANUARY 4, 2021","thank you, madam speaker. as we begin the 117..."
2,https://www.c-span.org/video/?507670-1&action=...,02:56:59,02:57:56,Mary Gay Scanlon,D-PA,"mary gay scanlon, d-pain the two years i have ...",116,"JANUARY 4, 2021","in the two years i have been here, every moti..."
3,https://www.c-span.org/video/?507670-1&action=...,03:02:03,03:03:13,Dan Newhouse,R-WA,"dan newhouse, r-wathank you, madam speaker. th...",116,"JANUARY 4, 2021","thank you, madam speaker. this rules package ..."
4,https://www.c-span.org/video/?507670-1&action=...,03:03:23,03:04:41,Sheila Jackson Lee,D-TX,"sheila jackson lee, d-txi thank the distinguis...",116,"JANUARY 4, 2021",i thank the distinguished chair and all of th...
...,...,...,...,...,...,...,...,...,...
3089,https://www.c-span.org/video/?525045-1&action=...,00:14:31,00:15:39,Bill Huizenga,R-MI,"bill huizenga, r-mimr. speaker, as the 117th c...",116,"DECEMBER 23, 2022","mr. speaker, as the 117th congress comes to a..."
3090,https://www.c-span.org/video/?525045-1&action=...,00:16:03,00:16:41,Tim Burchett,R-TN,"tim burchett, r-tnmr. speaker, i rise in stron...",116,"DECEMBER 23, 2022","mr. speaker, i rise in strong opposition to h..."
3091,https://www.c-span.org/video/?525045-1&action=...,00:17:45,00:38:25,Steny Hoyer,D-MD,"steny hoyer, d-mdrevise and extend my remarks....",116,"DECEMBER 23, 2022","revise and extend my remarks. mr. speaker, we..."
3092,https://www.c-span.org/video/?525045-1&action=...,00:39:07,00:39:39,Alex Mooney,R-WV,"alex mooney, r-wvthank you, mr. speaker. i ris...",116,"DECEMBER 23, 2022","thank you, mr. speaker. i rise today to share..."
